In [2]:
from EEGModels import EEGNet
import scipy
import numpy as np
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import balanced_accuracy_score

In [3]:
folder_path = './execution_data/'
file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.mat')]

file_list.sort()

print(file_list)

X = []
Y = []

for i, file in enumerate(file_list):
    x = scipy.io.loadmat(file)
    x = x['data']

    print(x.shape)

    first_letter = file.split('/')[-1][0]

    # create output vector. If file begins with 'P', then y = [1,0], else y = [0,1], having the same dimension as x
    if first_letter == 'P':
        y = np.zeros((x.shape[0],2))
        y[:,0] = 1
    else:
        y = np.zeros((x.shape[0],2))
        y[:,1] = 1

    X.append(x)
    Y.append(y)

f = open('data_sizes.txt', 'a')

f.write('walk_exec_raw: ' + str(sum([x.shape[0] for x in X])) + '\n')

f.close()

['./execution_data/P_1.mat', './execution_data/P_10.mat', './execution_data/P_2.mat', './execution_data/P_3.mat', './execution_data/P_4.mat', './execution_data/P_5.mat', './execution_data/P_9.mat', './execution_data/S_1.mat', './execution_data/S_10.mat', './execution_data/S_2.mat', './execution_data/S_3.mat', './execution_data/S_4.mat', './execution_data/S_5.mat', './execution_data/S_9.mat']
(84, 14, 1000)
(55, 14, 1000)
(74, 14, 1000)
(84, 14, 1000)
(86, 14, 1000)
(87, 14, 1000)
(90, 14, 1000)
(90, 14, 1000)
(98, 14, 1000)
(90, 14, 1000)
(120, 14, 1000)
(118, 14, 1000)
(97, 14, 1000)
(124, 14, 1000)


In [3]:

kernels, chans, samples = 1, 14, 1000

Accuracies = []
p_acc = []
s_acc = []
for index_1 in range(0, 7):
    for index_2 in range(7,14):
        # build training set from allfiles expect the ones with index_1 and index_2
        X_train = [X[i] for i in range(len(X)) if i != index_1 and i != index_2]
        Y_train = [Y[i] for i in range(len(Y)) if i != index_1 and i != index_2]

        # build test set from the files with index_1 and index_2
        X_test = [X[i] for i in range(len(X)) if i == index_1 or i == index_2]
        Y_test = [Y[i] for i in range(len(Y)) if i == index_1 or i == index_2]

        X_train = np.concatenate(X_train)
        Y_train = np.concatenate(Y_train)

        # print(Y_train[0], Y_train[1])

        # convert data to NHWC (trials, channels, samples, kernels) format.
        X_train = X_train.reshape(X_train.shape[0], chans, samples, kernels)

        model = EEGNet(nb_classes=2, Chans=chans, Samples=samples, dropoutRate=0.5, kernLength=32, F1=8, D=2, F2=16, dropoutType='Dropout')

        # compile the model and set the optimizers
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

        # count number of parameters in the model
        numParams    = model.count_params()     

        # set a valid path for the system to record model checkpoints
        # checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1, save_best_only=True)

        class_weights = {0:1, 1:1}

        fitted_model = model.fit(X_train, Y_train, batch_size = 32, epochs = 50, verbose = 2, 
                                class_weight=class_weights)
        
        #open file to write results in
        f = open("results_walk_exec_raw.txt", "a") 

        # make prediction on the 2 test files individually
        X_test_1 = X_test[0].reshape(X_test[0].shape[0], chans, samples, kernels)
        X_test_2 = X_test[1].reshape(X_test[1].shape[0], chans, samples, kernels)

        probs_1 = model.predict(X_test_1)
        probs_2 = model.predict(X_test_2)
        preds_1 = probs_1.argmax(axis = -1)
        preds_2 = probs_2.argmax(axis = -1)
        acc_1 = balanced_accuracy_score(Y_test[0].argmax(axis=-1), preds_1)
        acc_2 = balanced_accuracy_score(Y_test[1].argmax(axis=-1), preds_2)
        f.write("----------------------SELECTED FILES: %d and %d---------------------\n" % (index_1, index_2))
        f.write("Classification accuracy for P: %f \n" % (acc_1))
        f.write("Classification accuracy for S: %f \n" % (acc_2))

        p_acc.append(acc_1)
        s_acc.append(acc_2)

        # make prediction on the 2 test files together
        X_test = np.concatenate(X_test)
        X_test = X_test.reshape(X_test.shape[0], chans, samples, kernels)
        Y_test = np.concatenate(Y_test)


        probs       = model.predict(X_test)
        preds       = probs.argmax(axis = -1)
        acc         = balanced_accuracy_score(Y_test.argmax(axis=-1), preds)

        f.write("Classification accuracy for P and S: %f \n" % (acc))
        f.write("\n")

        f.close()

        Accuracies.append(acc)

print("Mean classification accuracy: %f " % (np.mean(Accuracies)))

# calculate the mean and std of the accuracies
p_acc = np.array(p_acc)
s_acc = np.array(s_acc)
acc = np.array(Accuracies)

f = open("results_walk_exec_raw.txt", "a")
f.write("Mean overall accuracy: %f \n" % (np.mean(acc)))
f.write("Std overall accuracy: %f \n" % (np.std(acc)))
f.write("Mean P accuracy: %f \n" % (np.mean(p_acc)))
f.write("Std P accuracy: %f \n" % (np.std(p_acc)))
f.write("Mean S accuracy: %f \n" % (np.mean(s_acc)))
f.write("Std S accuracy: %f \n" % (np.std(s_acc)))
f.close()



    

Epoch 1/50
36/36 - 13s - loss: 0.6754 - accuracy: 0.6055 - 13s/epoch - 364ms/step
Epoch 2/50
36/36 - 9s - loss: 0.6096 - accuracy: 0.6679 - 9s/epoch - 255ms/step
Epoch 3/50
36/36 - 9s - loss: 0.5748 - accuracy: 0.7088 - 9s/epoch - 254ms/step
Epoch 4/50
36/36 - 9s - loss: 0.5708 - accuracy: 0.7044 - 9s/epoch - 253ms/step
Epoch 5/50
36/36 - 9s - loss: 0.5428 - accuracy: 0.7435 - 9s/epoch - 261ms/step
Epoch 6/50
36/36 - 9s - loss: 0.5235 - accuracy: 0.7462 - 9s/epoch - 249ms/step
Epoch 7/50
36/36 - 9s - loss: 0.5177 - accuracy: 0.7516 - 9s/epoch - 254ms/step
Epoch 8/50
36/36 - 9s - loss: 0.5024 - accuracy: 0.7587 - 9s/epoch - 251ms/step
Epoch 9/50
36/36 - 9s - loss: 0.4778 - accuracy: 0.8023 - 9s/epoch - 248ms/step
Epoch 10/50
36/36 - 9s - loss: 0.4404 - accuracy: 0.8130 - 9s/epoch - 252ms/step
Epoch 11/50
36/36 - 9s - loss: 0.4122 - accuracy: 0.8468 - 9s/epoch - 253ms/step
Epoch 12/50
36/36 - 9s - loss: 0.3994 - accuracy: 0.8290 - 9s/epoch - 251ms/step
Epoch 13/50
36/36 - 9s - loss: 0.37

c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


6/6 [==============================] - 0s 49ms/step
Epoch 1/50
35/35 - 12s - loss: 0.6724 - accuracy: 0.5857 - 12s/epoch - 347ms/step
Epoch 2/50
35/35 - 9s - loss: 0.6160 - accuracy: 0.6601 - 9s/epoch - 264ms/step
Epoch 3/50
35/35 - 9s - loss: 0.5952 - accuracy: 0.6852 - 9s/epoch - 256ms/step
Epoch 4/50
35/35 - 9s - loss: 0.5652 - accuracy: 0.7121 - 9s/epoch - 256ms/step
Epoch 5/50
35/35 - 9s - loss: 0.5428 - accuracy: 0.7417 - 9s/epoch - 258ms/step
Epoch 6/50
35/35 - 9s - loss: 0.5132 - accuracy: 0.7794 - 9s/epoch - 255ms/step
Epoch 7/50
35/35 - 9s - loss: 0.4758 - accuracy: 0.8108 - 9s/epoch - 256ms/step
Epoch 8/50
35/35 - 9s - loss: 0.4105 - accuracy: 0.8350 - 9s/epoch - 262ms/step
Epoch 9/50
35/35 - 9s - loss: 0.3550 - accuracy: 0.8753 - 9s/epoch - 261ms/step
Epoch 10/50
35/35 - 9s - loss: 0.2891 - accuracy: 0.9031 - 9s/epoch - 250ms/step
Epoch 11/50
35/35 - 10s - loss: 0.2592 - accuracy: 0.9265 - 10s/epoch - 273ms/step
Epoch 12/50
35/35 - 9s - loss: 0.2428 - accuracy: 0.9238 - 9s/

c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


7/7 [==============================] - 0s 50ms/step
Epoch 1/50
35/35 - 13s - loss: 0.6768 - accuracy: 0.5869 - 13s/epoch - 363ms/step
Epoch 2/50
35/35 - 10s - loss: 0.6027 - accuracy: 0.6828 - 10s/epoch - 274ms/step
Epoch 3/50
35/35 - 9s - loss: 0.5641 - accuracy: 0.7195 - 9s/epoch - 266ms/step
Epoch 4/50
35/35 - 9s - loss: 0.5234 - accuracy: 0.7545 - 9s/epoch - 270ms/step
Epoch 5/50
35/35 - 9s - loss: 0.4699 - accuracy: 0.8047 - 9s/epoch - 271ms/step
Epoch 6/50
35/35 - 10s - loss: 0.4190 - accuracy: 0.8441 - 10s/epoch - 275ms/step
Epoch 7/50
35/35 - 10s - loss: 0.3749 - accuracy: 0.8719 - 10s/epoch - 277ms/step
Epoch 8/50
35/35 - 11s - loss: 0.3354 - accuracy: 0.8808 - 11s/epoch - 304ms/step
Epoch 9/50
35/35 - 10s - loss: 0.2900 - accuracy: 0.9131 - 10s/epoch - 283ms/step
Epoch 10/50
35/35 - 10s - loss: 0.2797 - accuracy: 0.9032 - 10s/epoch - 273ms/step
Epoch 11/50
35/35 - 10s - loss: 0.2355 - accuracy: 0.9301 - 10s/epoch - 275ms/step
Epoch 12/50
35/35 - 10s - loss: 0.2019 - accuracy:

...

7/7 [==============================] - 0s 53ms/step
Epoch 1/50
35/35 - 12s - loss: 0.6722 - accuracy: 0.5775 - 12s/epoch - 338ms/step
Epoch 2/50
35/35 - 9s - loss: 0.5968 - accuracy: 0.6946 - 9s/epoch - 252ms/step
Epoch 3/50
35/35 - 9s - loss: 0.5282 - accuracy: 0.7486 - 9s/epoch - 258ms/step
Epoch 4/50
35/35 - 9s - loss: 0.4892 - accuracy: 0.7586 - 9s/epoch - 254ms/step
Epoch 5/50
35/35 - 9s - loss: 0.4226 - accuracy: 0.8369 - 9s/epoch - 257ms/step
Epoch 6/50
35/35 - 9s - loss: 0.3951 - accuracy: 0.8396 - 9s/epoch - 255ms/step
Epoch 7/50
35/35 - 9s - loss: 0.3464 - accuracy: 0.8622 - 9s/epoch - 253ms/step
Epoch 8/50
35/35 - 9s - loss: 0.2936 - accuracy: 0.8919 - 9s/epoch - 251ms/step
Epoch 9/50
35/35 - 9s - loss: 0.2600 - accuracy: 0.9126 - 9s/epoch - 256ms/step
Epoch 10/50
35/35 - 9s - loss: 0.2469 - accuracy: 0.9234 - 9s/epoch - 255ms/step
Epoch 11/50
35/35 - 9s - loss: 0.2634 - accuracy: 0.9027 - 9s/epoch - 254ms/step
Epoch 12/50
35/35 - 9s - loss: 0.2583 - accuracy: 0.9018 - 9s/ep

c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


6/6 [==============================] - 0s 48ms/step
Epoch 1/50
34/34 - 12s - loss: 0.6873 - accuracy: 0.5614 - 12s/epoch - 341ms/step
Epoch 2/50
34/34 - 9s - loss: 0.6370 - accuracy: 0.6537 - 9s/epoch - 254ms/step
Epoch 3/50
34/34 - 9s - loss: 0.5921 - accuracy: 0.7027 - 9s/epoch - 252ms/step
Epoch 4/50
34/34 - 9s - loss: 0.5491 - accuracy: 0.7267 - 9s/epoch - 254ms/step
Epoch 5/50
34/34 - 9s - loss: 0.5143 - accuracy: 0.7618 - 9s/epoch - 253ms/step
Epoch 6/50
34/34 - 9s - loss: 0.4758 - accuracy: 0.7904 - 9s/epoch - 251ms/step
Epoch 7/50
34/34 - 9s - loss: 0.4336 - accuracy: 0.8190 - 9s/epoch - 257ms/step
Epoch 8/50
34/34 - 9s - loss: 0.3834 - accuracy: 0.8652 - 9s/epoch - 257ms/step
Epoch 9/50
34/34 - 8s - loss: 0.3399 - accuracy: 0.8587 - 8s/epoch - 250ms/step
Epoch 10/50
34/34 - 9s - loss: 0.2817 - accuracy: 0.9114 - 9s/epoch - 257ms/step
Epoch 11/50
34/34 - 9s - loss: 0.2228 - accuracy: 0.9326 - 9s/epoch - 253ms/step
Epoch 12/50
34/34 - 9s - loss: 0.1906 - accuracy: 0.9455 - 9s/ep

c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


7/7 [==============================] - 0s 51ms/step
Mean classification accuracy: 0.825793 
